# Exercise 1.1 - Importing Points of Interest from OpenStreetMaps and General Transit Feed Specification data


#### First we pull data from OpenStreetMaps using the [overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API). The query returns JSON which can either be loaded into the SAP HANA JSON Document Store, or "flattened" and loaded into a table. Both variants make use of a [hana-ml dataframe](https://help.sap.com/doc/1d0ebfe5e8dd44d09606814d83308d4b/latest/en-US/hana_ml.dataframe.html).

In [1]:
# import required libraries
import os
import pandas as pd
import requests
import hana_ml
from hana_ml.dataframe import ConnectionContext

In [13]:
# Connect to SAP HANA Cloud
host = '[YourHostName]'
port = 443
user = '[YourUser]'
passwd = '[YourUserPassword]'
cc= ConnectionContext(address=host, port=port, user=user, password=passwd, encrypt='true' ,sslValidateCertificate='false')

schema="TECHED_USER_000"
print('HANA version:', cc.hana_version())
print('hana-ml version:', hana_ml.__version__)
print('pandas version:', pd.__version__)

HANA version: 4.00.000.00.1663064250 (fa/CE2022.30)
hana-ml version: 2.14.22092300
pandas version: 1.2.5


In [4]:
# Get cafe|restaurant|bar amenities from 500m around Adelaide's center
# This query pulls a rather small amount of data from OSM
overpass_query = """
    [out:json];
    (
        node[amenity=cafe](around:500, -34.927975, 138.601394);
        node[amenity=restaurant](around:500, -34.927975, 138.601394);
        node[amenity=bar](around:500, -34.927975, 138.601394);
    );
    out geom;
"""
overpass_url = "http://overpass-api.de/api/interpreter"
response = requests.get(overpass_url, params={'data': overpass_query})
data_small = response.json()

In [5]:
# Inspect the result
data_small['elements'][0]

{'type': 'node',
 'id': 267954364,
 'lat': -34.9245258,
 'lon': 138.599892,
 'tags': {'amenity': 'cafe',
  'brand': 'Cibo Espresso',
  'brand:wikidata': 'Q5119230',
  'brand:wikipedia': 'en:Cibo Espresso',
  'cuisine': 'coffee_shop',
  'name': 'Cibo Espresso',
  'name:en': 'Cibo Espresso',
  'opening_hours': 'Mo-Fr 07:00-17:00; Sa 07:00-14:00; Su 08:00-12:00',
  'phone': '+61 8 8410 4088',
  'postal_code': '5000',
  'takeaway': 'yes'}}

In [6]:
# The overpass API resturns JSON which we can store in the HANA Document Store.
from hana_ml.docstore import create_collection_from_elements
coll = create_collection_from_elements(
    connection_context = cc,
    schema = schema,
    collection_name = 'POI_COLLECTION_SMALL',
    elements = data_small["elements"], 
    drop_exist_coll = True)

In [7]:
# As an alternative, we can also flatten the JSON data into a pandas dataframe
df_small = pd.json_normalize(data_small, record_path =['elements'])
df_small = df_small[['id','type','lon','lat','tags.amenity','tags.name']]
df_small.head(5)

,id,type,lon,lat,tags.amenity,tags.name
0,267954364,node,138.599892,-34.924526,cafe,Cibo Espresso
1,389349967,node,138.605437,-34.925643,cafe,Bean Bar
2,389351668,node,138.603446,-34.925728,cafe,Cibo Espresso
3,389353056,node,138.602958,-34.927201,cafe,Lena's
4,585135535,node,138.600485,-34.932359,restaurant,La Trattoria


In [8]:
# ... and store the flattened data in a HANA table
from hana_ml.dataframe import create_dataframe_from_pandas
hdf_pois_small = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df_small, 
    schema=schema,
    table_name='POIS_SMALL', 
    geo_cols=[("lon", "lat")], srid=4326,
    primary_key=('id'), allow_bigint=True,
    drop_exist_tab=True, force=True
    )

100%|██████████| 1/1 [00:00<00:00,  5.40it/s]


#### Next we import Adelaide's GTFS data
https://gtfs.adelaidemetro.com.au/v1/static/latest/google_transit.zip or <br>
https://storage.googleapis.com/storage/v1/b/mdb-latest/o/au-south-australia-adelaide-metro-gtfs-660.zip?alt=media

The zip archive contains a couple of files. In this demo we import just a subset. First we read the .txt files as pandas dataframes.

In [11]:
p = 'C:/Users/D003411/Documents/GitHub/teched2022-DA180/data/gtfs/adelaide';

#path = os.path.join(p, 'agency.txt')
#df_agency = pd.read_csv(path)
#path = os.path.join(p, 'calendar.txt')
#df_calendar = pd.read_csv(path)
#path = os.path.join(p, 'calendar_dates.txt')
#df_calendarddates = pd.read_csv(path)
#path = os.path.join(p, 'frequencies.txt')
#df_frequencies = pd.read_csv(path)
#path = os.path.join(p, 'pathways.txt')
#df_pathways = pd.read_csv(path)
path = os.path.join(p, 'routes.txt')
df_routes = pd.read_csv(path)
path = os.path.join(p, 'shapes.txt')
df_shapes = pd.read_csv(path, low_memory=False)
path = os.path.join(p, 'stop_times.txt')
df_stoptimes = pd.read_csv(path, low_memory=False)
path = os.path.join(p, 'stops.txt')
df_stops = pd.read_csv(path)
path = os.path.join(p, 'transfers.txt')
df_transfers = pd.read_csv(path)
path = os.path.join(p, 'trips.txt')
df_trips = pd.read_csv(path)

Thne we upload the data from the pandas dataframes into a HANA table using hana-ml.
The nice thing is that pandas guesse the datatypes pretty good, and using hana-ml you can create a spatial table column directly from the lon/lat coordinates.<br>
<i>geo_cols=[("stop_lon", "stop_lat")], srid=4326</i>

In [12]:
from hana_ml.dataframe import create_dataframe_from_pandas
hdf_routes = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df_routes, 
    schema=schema, drop_exist_tab=True, 
    table_name='GTFS_ROUTES', force=True,
    primary_key='route_id'
    )
hdf_stops = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df_stops, 
    schema=schema, drop_exist_tab=True, 
    table_name='GTFS_STOPS', force=True, allow_bigint=True,
    geo_cols=[("stop_lon", "stop_lat")], srid=4326,
    primary_key='stop_id'
    )
hdf_transfers = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df_transfers, 
    schema=schema, drop_exist_tab=True, 
    table_name='GTFS_TRANSFERS', force=True,
    primary_key=('from_stop_id', 'to_stop_id')
    )
hdf_trips = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df_trips, 
    schema=schema, drop_exist_tab=True, 
    table_name='GTFS_TRIPS', force=True,
    primary_key='trip_id'
    )
hdf_shapes = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df_shapes, 
    schema=schema, drop_exist_tab=True, 
    table_name='GTFS_SHAPES', force=True,
    geo_cols=[("shape_pt_lon", "shape_pt_lat")], srid=4326,
    primary_key=('shape_id','shape_pt_sequence')
    )
hdf_stoptimes = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df_stoptimes, 
    schema=schema, drop_exist_tab=True, 
    table_name='GTFS_STOPTIMES', force=True,
    primary_key=('trip_id', 'stop_sequence')
    )

100%|██████████| 17/17 [01:45<00:00,  6.21s/it]
